In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import gc

# 1. set file path

In [2]:
print(os.getcwd())
path = r'/Users/zoe/Desktop/ptt_crawler/'
exec_dt = datetime.strftime(datetime.now(), '%Y%m%d_%H%M%S')
os.listdir(path)

/Users/zoe/Desktop/ptt_crawler/local_script


['Gossiping-1-5.json',
 'Gossiping-39361-39363_reply_20191228_191753.csv',
 'ntusd-negative.txt',
 'ptt_id_check.ipynb',
 '.DS_Store',
 'Gossiping-28800-29600.json',
 'ptt_excel.xlsx',
 'Gossiping-1-5_article_20191228_201931.csv',
 'Gossiping-1-5_article_20191228_191753.csv',
 'local_script',
 'neo4j_community_pagerank_1204-20191211T114233Z-001.zip',
 'python -m PttWebCrawler -b PublicServan -i 100 200',
 'Gossiping-39361-39363.json',
 'test.json',
 'Gossiping-37500-39075.json',
 'ntusd-positive.txt',
 'neo4j_community_pagerank_1204',
 'Gossiping-39361-39363_article_20191228_191753.csv',
 '.ipynb_checkpoints',
 'Gossiping-37700-39294_1207.json',
 'test2.json',
 'Gossiping-1-5_reply_20191228_201931.csv',
 'Gossiping-1-5_reply_20191228_191753.csv',
 'HatePolitics-2600-4035.json']

# 2. check json format

In [3]:
def check_json_format(file):
    with open(file, 'ab+') as f:
        # read as binary is for reverse file search
        print('check trivial comma in EOF before revesing')
        # * 2 -- end of stream; offset is usually negative
        f.seek(-6, 2)
  
        if f.read(200) == b'}]},]}':
            print('revising error endings')
            f.seek(-3, 2)
            f.truncate()
            f.write(b']}')
  
        print('check trivial comma in EOF after revising')
        f.seek(-20, 2)
        print(f.read(200))

# 3. parse json file
## notes:
#### 1. bug of special character in nickname
#### ex: rex520368 (b@N9) https://www.ptt.cc/bbs/Lifeismoney/M.1575016206.A.74B.html
#### check method: art_df[art_df.author == 'NoData'] 
#### 2. remove filter feature: (1) no reply (2) unknow author

In [4]:
def ptt_json_to_df(ptt_df):
    art_df = []
    #currently ignore error records
    invalid_lst = []
    
    for x in ptt_df.itertuples():
        #some inconsistent page failed!
        if x.articles == {'error': 'invalid url'}:
            print('invalid record')
            print(x.Index)
            invalid_lst.append(x)
            continue
        art = pd.DataFrame([x.articles])
        
        if art.author[0] is None:
            art.author = 'NoData'    
        else:
            art.author = art.author.replace(r'\([^)]*\)', '',regex=True).values[0].replace(' ', '')
        
        #fixing package typo of columns
        art.rename({'message_conut':'message_count'}, axis=1, inplace=True)
        reply_aggr = pd.DataFrame(art.message_count.values[0], index=[0])
        art.drop(['message_count'], axis=1, inplace=True)
        art = pd.concat([art, reply_aggr], axis=1)
        art_df.append(art)
    art_df = pd.concat(art_df)
    art_df.reset_index(drop=True, inplace=True)
    return art_df

In [5]:
def ptt_art_parser(f_nm):
    ##this parser is for jwlin crawler's result
    print('{} parsing start!'.format(f_nm))
    ptt_df = pd.read_json(f_nm, encoding='utf8')    
    ptt_df_t1 = ptt_json_to_df(ptt_df)
    ptt_df_t1.date = pd.to_datetime(ptt_df_t1.date, format='%a %b %d %H:%M:%S %Y', errors='coerce')
    print('parsing finished!')
    return ptt_df_t1

# 4. reply parsing
## fix this incompleted parser.....

In [6]:
def ptt_reply_parser(art_df):
    reply_df = []
    pct_20 = int(art_df.shape[0]/20)
    print('Total iter is {}'.format(art_df.shape[0]))

    for idx, x in enumerate(art_df.itertuples()):
        if idx % pct_20 == 0:
            print(idx)
            print(datetime.now())
            gc.collect()

        if x.messages == []:
            continue
        tmp_reply = pd.DataFrame(x.messages)
        tmp_reply['ip'] = tmp_reply.push_ipdatetime.str.split(' ', n=1, expand=True)[0]
        tmp_ip_dt = tmp_reply.push_ipdatetime.copy()
        tmp_dt = np.where(tmp_ip_dt.str.len() < 20, tmp_ip_dt, tmp_ip_dt.str[-11:])

        #add ip....
        if type(x.date) is not str:
            tmp_reply['reply_datetime'] = str(x.date.year) + '/' + tmp_dt
        else:
            tmp_reply['reply_datetime'] = tmp_dt
        ##以發文日期作為推文的估計值，因為push_ipdatetime的ip 跟 datetime放在一起，有些只有datetime 沒有ip，好像跟RE 有關....
        ##欄位重新命名，push_userid 改為 source

        tmp_reply['author'] = x.author
        tmp_reply['article_id'] = x.article_id
        tmp_reply['board'] = x.board
        tmp_reply['date'] = x.date

        reply_df.append(tmp_reply)
    reply_df = pd.concat(reply_df)
    return reply_df

# 5. intergrate whole function

In [7]:
def intg_parser(path, file_nm):
    file_path = path + file_nm
    check_json_format(file_path)
    art_df = ptt_art_parser(file_path)
    art_df['re_flag'] = np.where(art_df.article_title.str.contains('Re'), 'Y', 'N')
    art_df['expl_flag'] = np.where(art_df['count'] >= 100, 'Y', 'N')
    #avoid comma parse error in csv
    art_df.replace(',', '，', regex=True, inplace=True)

    #selected output article
    slc_col = ['article_id', 'article_title', 'author', 'board', 'content', 'date',
               'ip', 'all', 'boo', 'count', 'neutral', 'push', 're_flag', 'expl_flag']
    #output article csv
    art_df[slc_col].to_csv(path + file_nm.strip('.json') + '_article_{}.csv'.format(exec_dt))

    #parsing reply
    reply_df = ptt_reply_parser(art_df)
    reply_df.reset_index(drop=True, inplace=True)
    reply_df.replace(',', '，', regex=True, inplace=True)
    reply_df[reply_df.push_content.str.contains(',')]

    #not transfer format to datetime(pd.to_datetime(df, errors='coerce')), because there are some elements without year.
    reply_df.reply_datetime.replace('/', '-', regex=True, inplace=True)
    #output reply csv
    reply_df.to_csv(path + file_nm.strip('.json') + '_reply_{}.csv'.format(exec_dt))

# 6. exec whole process

In [8]:
for file in os.listdir(path):
    if file.endswith('json'):
        print(file)
        intg_parser(path=path, file_nm=file)

Gossiping-1-5.json
check trivial comma in EOF before revesing
check trivial comma in EOF after revising
b' "alligator176"}]}]}'
/Users/zoe/Desktop/ptt_crawler/Gossiping-1-5.json parsing start!
parsing finished!
Total iter is 100
0
2019-12-28 20:19:32.244067
5
2019-12-28 20:19:32.296363
10
2019-12-28 20:19:32.340842
15
2019-12-28 20:19:32.387559
20
2019-12-28 20:19:32.428973
25
2019-12-28 20:19:32.473862
30
2019-12-28 20:19:32.517379
35
2019-12-28 20:19:32.561865
40
2019-12-28 20:19:32.605348
45
2019-12-28 20:19:32.643809
50
2019-12-28 20:19:32.685874
55
2019-12-28 20:19:32.730852
60
2019-12-28 20:19:32.778437
65
2019-12-28 20:19:32.817650
70
2019-12-28 20:19:32.856731
75
2019-12-28 20:19:32.896824
80
2019-12-28 20:19:32.937059
85
2019-12-28 20:19:32.980677
90
2019-12-28 20:19:33.024373
95
2019-12-28 20:19:33.069943
